In [1]:
import asyncio
import atexit
import threading
import time
from typing import List
from enum import Enum

import grpc
from google.protobuf.empty_pb2 import Empty

from reachy_sdk_api import camera_reachy_pb2_grpc
from reachy_sdk_api import joint_pb2, joint_pb2_grpc
from reachy_sdk_api import fan_pb2_grpc
from reachy_sdk_api import sensor_pb2, sensor_pb2_grpc
from reachy_sdk_api import restart_signal_pb2, restart_signal_pb2_grpc

from reachy_sdk_api.gripperMX28_pb2 import GripperMX28Command, GrippersMX28Command
from reachy_sdk_api.gripper_pb2 import GripperCommand, GripperId, GrippersAck
from reachy_sdk_api.gripperMX28_pb2_grpc import GripperMX28ServiceStub

In [2]:
grpc_channel = grpc.insecure_channel(f'10.42.0.1:50055')

Gripper stuff

In [3]:
gripper_stub = GripperMX28ServiceStub(grpc_channel)

## Checking compliance

In [10]:
for i in range(50) :
    gripper_stub.SendGripperMX28Commands(GrippersMX28Command(commands=[GripperMX28Command(id=GripperId.RIGHT, goal_position=0.35)]))
    time.sleep(0.01)

In [24]:
reachy.head.look_at(0.5,-0.3,0.0,duration=3.0)

Lift both arms, put them in stiff mode and check if *turn_off_smoothly* works.

In [17]:
reachy.turn_on('reachy')

In [18]:
reachy.turn_off_smoothly('reachy')

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/remi/reachy-sdk/reachy_sdk/reachy_sdk.py", line 185, in _start_sync_in_bg
    loop.run_until_complete(self._sync_loop())
  File "/usr/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
    return future.result()
  File "/home/remi/reachy-sdk/reachy_sdk/reachy_sdk.py", line 240, in _sync_loop
    await asyncio.gather(
  File "/home/remi/reachy-sdk/reachy_sdk/reachy_sdk.py", line 208, in _get_stream_sensor_loop
    async for state_update in stub.StreamSensorStates(stream_req):
  File "/home/remi/.virtualenvs/reachy/lib/python3.8/site-packages/grpc/aio/_call.py", line 326, in _fetch_stream_responses
    await self._raise_for_status()
  File "/home/remi/.virtualenvs/reachy/lib/python3.8/site-packages/grp

## Moving the arms

Put each arm at 90 degrees.

In [19]:
try:
    right_angled_position_right = {
    reachy.r_arm.r_shoulder_pitch: 0,
    reachy.r_arm.r_shoulder_roll: 0,
    reachy.r_arm.r_arm_yaw: 0,
    reachy.r_arm.r_elbow_pitch: -90,
    reachy.r_arm.r_forearm_yaw: 0,
    reachy.r_arm.r_wrist_pitch: 0,
    reachy.r_arm.r_wrist_roll: 0,
}
except AttributeError:
    pass

try:
    right_angled_position_left = {
        reachy.l_arm.l_shoulder_pitch: 0,
        reachy.l_arm.l_shoulder_roll: 0,
        reachy.l_arm.l_arm_yaw: 0,
        reachy.l_arm.l_elbow_pitch: -90,
        reachy.l_arm.l_forearm_yaw: 0,
        reachy.l_arm.l_wrist_pitch: 0,
        reachy.l_arm.l_wrist_roll: 0,
    }
except AttributeError:
    pass

In [14]:
reachy.turn_on('reachy')

In [40]:
try:
    goto(
        goal_positions=right_angled_position_right,
        duration=1.0,
        interpolation_mode=InterpolationMode.LINEAR
    )
    goto(
        goal_positions=right_angled_position_left,
        duration=1.0,
        interpolation_mode=InterpolationMode.MINIMUM_JERK
    )
except NameError:
    print('Reachy has no right arm!')

In [11]:
try:
    goto(
        goal_positions=right_angled_position_left,
        duration=1.0,
        interpolation_mode=InterpolationMode.MINIMUM_JERK
    )
except NameError:
    print('Reachy has no left arm!')

In [41]:
reachy.turn_off_smoothly('reachy')

Reproduce the square movement from [the documentation](https://pollen-robotics.github.io/reachy-2021-docs/sdk/first-moves/kinematics/#inverse-kinematics), to check if this Reachy can reproduce it.

For the right arm:

In [42]:
A = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.4],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

B = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.4],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

C = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.1],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

D = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.1],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

In [43]:
try:
    joint_pos_A = reachy.r_arm.inverse_kinematics(A)
    joint_pos_B = reachy.r_arm.inverse_kinematics(B)
    joint_pos_C = reachy.r_arm.inverse_kinematics(C)
    joint_pos_D = reachy.r_arm.inverse_kinematics(D)
except AttributeError:
    print('Reachy has no right arm!')

In [44]:
import time
# put the joints in stiff mode
try:
    reachy.turn_on('r_arm')

    # use the goto function
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_A)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_B)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_C)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_D)}, duration=1.0)

    # put the joints back to compliant mode
    # use turn_off_smoothly to prevent the arm from falling hard
    reachy.turn_off_smoothly('r_arm')
    
except AttributeError:
    print('Reachy has no right arm!')

For the left arm:

In [45]:
A = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.1],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

B = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.1],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

C = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.3],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

D = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.3],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

In [46]:
try:
    joint_pos_A = reachy.l_arm.inverse_kinematics(A)
    joint_pos_B = reachy.l_arm.inverse_kinematics(B)
    joint_pos_C = reachy.l_arm.inverse_kinematics(C)
    joint_pos_D = reachy.l_arm.inverse_kinematics(D)
except AttributeError:
    print('Reachy has no left arm')

In [47]:
# put the joints in stiff mode
try:
    reachy.turn_on('l_arm')

    # use the goto function
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_A)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_B)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_C)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_D)}, duration=1.0)

    # put the joints back to compliant mode
    # use turn_off_smoothly to prevent the arm from falling hard
    reachy.turn_off_smoothly('l_arm')
except AttributeError:
    print('Reachy has no left arm!')

## Checking the force sensors

You should see one force sensor per arm, zith the *.force_sensors* object.

In [51]:
reachy.force_sensors

<Holder
	<ForceSensor name="l_force_gripper" force="207.92">
>

The force value should be positive when applying a force towards the exterior of the gripper (e.g. when you try to open the gripper which is in stiff mode). On the contrary, you should get a negative force value when applying a force towards the interior of the gripper.  

First, place the grippers in stiff mode, it will be easier to check the force values.

In [52]:
try:
    reachy.l_arm.l_gripper.compliant = False
except AttributeError:
    print('Reachy has no left arm.')

In [53]:
try:
    reachy.r_arm.r_gripper.compliant = False
except AttributeError:
    print('Reachy has no right arm.')

Now, try to close each gripper and check that when you doing this, the force value is negative.

In [54]:
reachy.force_sensors.r_force_gripper

AttributeError: 'DeviceHolder' object has no attribute 'r_force_gripper'

In [55]:
reachy.force_sensors.l_force_gripper

<ForceSensor name="l_force_gripper" force="-93.33">

Next, try to open each gripper and check that when you doing this, the force value is positive.

In [56]:
reachy.force_sensors.r_force_gripper

AttributeError: 'DeviceHolder' object has no attribute 'r_force_gripper'

In [57]:
reachy.force_sensors.l_force_gripper

<ForceSensor name="l_force_gripper" force="211.92">

Finally, put the grippers back to compliant mode.

In [58]:
try:
    reachy.l_arm.l_gripper.compliant = True
except AttributeError:
    pass

In [59]:
try:
    reachy.r_arm.r_gripper.compliant = True
except AttributeError:
    pass